In [1]:
import warnings
warnings.filterwarnings('ignore')

# 1. Base case tokenization

In [2]:
from transformers import DistilBertTokenizer

sentence = 'happy'

tokenizer_ckpt = 'distilbert/distilbert-base-uncased-finetuned-sst-2-english'
tokenizer = DistilBertTokenizer.from_pretrained(tokenizer_ckpt)

In [3]:
tokenizer.tokenize(sentence)

['happy']

# 2. Encryption (AES-256)

In [4]:
import os
import base64

from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.padding import PKCS7
from cryptography.hazmat.backends import default_backend

In [5]:
def generate_key(password: str, salt: bytes) -> bytes:
    kdf = PBKDF2HMAC(
        algorithm=hashes.SHA256(),
        length=32,
        salt=salt,
        iterations=100000,
        backend=default_backend()
    )
    return kdf.derive(password.encode())

In [6]:
password = "securepassword"
salt = os.urandom(16)
iv = os.urandom(16)

key = generate_key(password, salt)
key

b"X\xd5\xf8\xcb$\x1e\x9d\x7f'\xaag+\xd3>1\xcb\xf8-0\x8b\x9d\xa8\xffa\xeeP\x0c-&\xaa3\xfc"

In [7]:
def encrypt(data: str, key: bytes, iv: bytes) -> str:
    cipher = Cipher(algorithms.AES(key), modes.CBC(iv), backend=default_backend())
    encryptor = cipher.encryptor()
    
    padder = PKCS7(128).padder()
    padded_data = padder.update(data.encode()) + padder.finalize()
    
    encrypted = encryptor.update(padded_data) + encryptor.finalize()
    return base64.b64encode(encrypted).decode()

In [8]:
plaintext = tokenizer.tokenize(sentence)[0]
plaintext

'happy'

In [9]:
encrypted = encrypt(plaintext, key, iv)
encrypted

'ogatzAKtNKEvabbzM7kUig=='

In [10]:
def decrypt(encrypted_data: str, key: bytes, iv: bytes) -> str:
    cipher = Cipher(algorithms.AES(key), modes.CBC(iv), backend=default_backend())
    decryptor = cipher.decryptor()
    
    encrypted_data = base64.b64decode(encrypted_data)
    decrypted = decryptor.update(encrypted_data) + decryptor.finalize()
    
    unpadder = PKCS7(128).unpadder()
    unpadded_data = unpadder.update(decrypted) + unpadder.finalize()
    
    return unpadded_data.decode()

In [12]:
decrypted = decrypt(encrypted, key, iv)
decrypted

'happy'